In [11]:
import os
import sys
from tqdm.notebook import tqdm 
import numpy as np
from numpy.linalg import eig
# from pprint import pprint
from google.colab import drive
# import copy
import pandas as pd
import math
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt


drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
col = list()
def read_data(fname):
    x = []
    y = []
    global col
    with open(fname, encoding = 'utf-8') as f:
      for line in f.readlines():
        sample = line.strip().split(',')
        x.append(sample[:-1])
        y.append(sample[-1])
    # print(sample)
    x = np.array(x).astype(np.float32)
    y  = np.array(y).astype(np.float32)
    y = ((y.astype(np.int32)-1)*2-1)
    print("(xshape, yshape): ",end='')
    print(x.shape, y.shape)
    col = [str(i+1) for i in range(x.shape[1])]
    col.append('class')
    print("Columns: ", end='')
    print(col)
    return np.hstack((x,y.reshape(-1,1)))

In [3]:
datafile = os.getcwd() + '/drive/My Drive/data/ps4/sonar_train.data'
dataset = read_data(datafile)
data = pd.DataFrame(data = dataset, columns=col)
# norm_t = Normalizer().fit(data[data.columns[:-1]])
# data[data.columns[:-1]] = norm_t.transform(data[data.columns[:-1]])


# mean = data.mean()
# std = data.std()
# mean[-1] = 0
# std[:] = 1
# data = (data-mean)/std

(xshape, yshape): (104, 60) (104,)
Columns: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', 'class']


In [4]:
datafile = os.getcwd() + '/drive/My Drive/data/ps4/sonar_valid.data'
valid_dataset = read_data(datafile)
valid_data = pd.DataFrame(data = valid_dataset, columns=col)

# valid_data = (valid_data-mean)/std

(xshape, yshape): (52, 60) (52,)
Columns: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', 'class']


In [5]:
datafile = os.getcwd() + '/drive/My Drive/data/ps4/sonar_test.data'
test_dataset = read_data(datafile)
test_data = pd.DataFrame(data = test_dataset, columns=col)
# test_data = (pd.DataFrame(data = test_dataset, columns=col)-mean)/std

(xshape, yshape): (52, 60) (52,)
Columns: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', 'class']


# Logistic regression

In [0]:
class Logistic_regression:
    def __init__(self, lr,data):
        # self.w = np.random.normal(size=(data.shape[1]-1))
        self.w = np.zeros(data.shape[1]-1)
        self.b = 0
        self.lr = lr
        # print(self.w)
        # print(self.b)
 
    def P(self, x, y):
        exp = np.exp(np.dot(x,self.w)+self.b)*y/(1+np.exp(np.dot(x,self.w)+self.b))
        return ((1-y)/2)+exp

    def predict(self, x):
        p_n1 = self.P(x,-1)
        p_p1 = self.P(x, 1)
        p = p_p1>=p_n1
        p = np.array(p).astype(np.int32)
        return p*2-1
    
    def accuracy(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        p = self.predict(x)
        # print(p)
        p = p*y
        return p[p>0].shape[0]/p.shape[0]

    def MLE(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        return np.sum(((y+1)/2)*np.dot(x,self.w+self.b) - np.log(1+np.exp(np.dot(x,self.w)+self.b)))

    def update_step(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        delta_b = np.sum(( (y+1)/2 ) - self.P( x, np.ones(x.shape[0]) ))
        delta_w = (( (y+1)/2 ) - self.P( x, np.ones(x.shape[0]) )) @ x 
        self.w = self.w + self.lr*delta_w
        self.b = self.b + self.lr*delta_b

In [0]:
clf = Logistic_regression(0.01, data)
abs(clf.MLE(data)),clf.accuracy(data)
clf.update_step(data)

In [8]:
(clf.w*clf.w).shape

(60,)

In [9]:
x = np.array(data[data.columns[:-1]])
y = np.array(data['class'])
# for i in range(x.shape[0]):
#     print("i = ",clf.P(x[i],y[i]))

clf.P(x,y)
clf.accuracy(data)

0.5384615384615384

# 3(a)

In [17]:
clf = Logistic_regression(0.05, data)
epochs = 80000
prev_l= 0
for i in tqdm(range(epochs), leave=False):
    clf.update_step(data)
    if(i%4000==0):
        print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)))
        # print(clf.w[:10])
    if(clf.accuracy(data)==1 or abs(clf.MLE(data)-prev_l)<0.001):
        print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
        print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
        break
    prev_l = clf.MLE(data)

Accuracy: 53.85, Likelihood Estimation: -29.320050013875566, delta loss: 29.320050013875566
Accuracy: 91.35, Likelihood Estimation: -9701.172535245361, delta loss: 193.72638236528292
Accuracy: 98.08, Likelihood Estimation: -10618.946038485761, delta loss: 1.3069674256494181
converged at iteration :8049 where delta loss: 0.00013321816913958173 
Accuracy on test data: 80.77%


# Logistic regression with Regularization

In [0]:
class Logistic_regression_L2:
    def __init__(self, lr, lbd, data):
        self.w = np.zeros(data.shape[1]-1)
        self.b = 0
        self.lbd = lbd
        self.lr = lr

    def P(self, x, y):
        exp = np.exp(np.dot(x,self.w)+self.b)*y/(1+np.exp(np.dot(x,self.w)+self.b))
        return ((1-y)/2)+exp

    def predict(self, x):
        p_n1 = self.P(x,-1)
        p_p1 = self.P(x, 1)
        p = p_p1>=p_n1
        p = np.array(p).astype(np.int32)
        return p*2-1
    
    def accuracy(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        p = self.predict(x)
        # print(p)
        p = p*y
        return p[p>0].shape[0]/p.shape[0]

    def MLE(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        return np.sum(((y+1)/2)*np.dot(x,self.w+self.b) - np.log(1+np.exp(np.dot(x,self.w)+self.b))) - np.sum((self.w*self.w))*self.lbd/2

    def update_step(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        delta_b = np.sum(( (y+1)/2 ) - self.P( x, np.ones(x.shape[0]) ))
        delta_w = (( (y+1)/2 ) - self.P( x, np.ones(x.shape[0]) )) @ x 
        print(delta_w)
        self.w = self.w + delta_w * self.lr - self.lbd * self.w * self.lr
        self.b = self.b + delta_b * self.lr

In [10]:
clf = Logistic_regression_L2(0.1, 0.0001, data)
clf.update_step(data)

[ 2.93900006e-01  2.84749989e-01  2.18600005e-01  3.66800003e-01
  2.67949984e-01  2.42799965e-01  2.83150024e-01  7.13200021e-01
  2.21295008e+00  2.76164995e+00  3.42495003e+00  3.17520003e+00
  2.93729999e+00  1.75354998e+00  9.93650035e-01  2.15050012e-01
 -2.17650013e-01  5.86099945e-01  2.65565008e+00  3.71710006e+00
  3.37789992e+00  2.87994994e+00  2.42190000e+00  1.53135005e+00
  5.37599960e-01  3.20699960e-01  9.24350020e-01  1.45165014e+00
  1.08215005e+00  6.34499952e-01 -3.44400018e-01  3.01550033e-01
 -5.17650014e-01 -1.73264985e+00 -2.96450001e+00 -3.87235004e+00
 -3.23359998e+00 -1.14585003e+00 -4.49981540e-04 -8.93449988e-01
 -7.96149999e-01  3.59399998e-01  1.66339994e+00  1.91080001e+00
  1.86520001e+00  1.20829999e+00  1.00930003e+00  8.90750032e-01
  4.87800003e-01  5.33499959e-02  2.08750005e-01  1.47500000e-01
  5.17000000e-02  5.96500005e-02  1.60000020e-02  3.84999995e-02
 -1.77499985e-02  5.59999997e-02  2.72500006e-02  2.09500020e-02]


In [0]:
clf = Logistic_regression_L2(0.1, 0.0001, data)
epochs = 1000000
prev_l= 0
for i in tqdm(range(epochs)):
    clf.update_step(data)
    if(i%4000==0 and  i>1000):
        print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)))
        # print(clf.w[:10])
        if(abs(clf.MLE(data)-prev_l)<0.1):
            print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
            break
        prev_l = clf.MLE(data)

In [38]:
clf.accuracy(test_data)

0.7884615384615384

In [46]:
epochs = 100000
lbd = [10.0**i for i in np.arange(-12,-3)]
for c in lbd:
    print("Lambda =  ",c)
    clf = Logistic_regression_L2(1, c, data)
    prev_l= 0
    for i in tqdm(range(epochs)):
        clf.update_step(data)
        if(i%4000==0 and i>1000):
            print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {:.2f}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)),end=' ')
            print("Validation accuracy: {:.2f}% at iter={}".format(100*clf.accuracy(valid_data), i))
            # print(clf.w[:10])
            if(abs(clf.MLE(data)-prev_l)<1 or i==epochs-1):
                print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
                print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
                print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
                break
            prev_l = clf.MLE(data)
        if(i==epochs-1):
            print("delta loss: {} with MLE: {}".format(abs(clf.MLE(data)-prev_l), clf.MLE(data)))
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))

Lambda =   1e-12


Accuracy: 84.62, Likelihood Estimation: -201472.55512519678, delta loss: 201472.56 Validation accuracy: 67.31% at iter=4000
Accuracy: 94.23, Likelihood Estimation: -204963.75930574577, delta loss: 3491.20 Validation accuracy: 82.69% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -215347.96926050237, delta loss: 10384.21 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -215653.54414141047, delta loss: 305.57 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -215851.24953059608, delta loss: 197.71 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -215999.68433664602, delta loss: 148.43 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -216117.60948835957, delta loss: 117.93 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -216215.08374562836, delta loss: 97.47 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.0

Accuracy: 91.35, Likelihood Estimation: -191848.6198140096, delta loss: 191848.62 Validation accuracy: 82.69% at iter=4000
Accuracy: 98.08, Likelihood Estimation: -207272.2563961435, delta loss: 15423.64 Validation accuracy: 75.00% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -219927.07141790385, delta loss: 12654.82 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -220321.97772428807, delta loss: 394.91 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -220572.0263412868, delta loss: 250.05 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -220750.01631006828, delta loss: 177.99 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -220885.63693727038, delta loss: 135.62 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -220994.37501989352, delta loss: 108.74 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00

Accuracy: 75.00, Likelihood Estimation: -206980.8088533685, delta loss: 206980.81 Validation accuracy: 69.23% at iter=4000
Accuracy: 94.23, Likelihood Estimation: -210499.12333581105, delta loss: 3518.31 Validation accuracy: 73.08% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -220361.40761870358, delta loss: 9862.28 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -220634.78905628098, delta loss: 273.38 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -220796.86230523227, delta loss: 162.07 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -220913.8989555197, delta loss: 117.04 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -221006.80688875815, delta loss: 92.91 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -221084.8002599341, delta loss: 77.99 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00, Li

Accuracy: 93.27, Likelihood Estimation: -197432.81728633647, delta loss: 197432.82 Validation accuracy: 75.00% at iter=4000
Accuracy: 98.08, Likelihood Estimation: -206516.13253101395, delta loss: 9083.32 Validation accuracy: 80.77% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -217782.36879913398, delta loss: 11266.24 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218037.9139688833, delta loss: 255.55 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -218188.81896661693, delta loss: 150.90 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -218299.78782183808, delta loss: 110.97 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -218389.95417110948, delta loss: 90.17 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -218466.97650624052, delta loss: 77.02 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00,

Accuracy: 74.04, Likelihood Estimation: -181173.67064222257, delta loss: 181173.67 Validation accuracy: 80.77% at iter=4000
Accuracy: 80.77, Likelihood Estimation: -198354.70767821374, delta loss: 17181.04 Validation accuracy: 86.54% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -218217.02125584744, delta loss: 19862.31 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218501.8594206514, delta loss: 284.84 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -218672.9186613848, delta loss: 171.06 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -218800.4594834633, delta loss: 127.54 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -218904.11726184058, delta loss: 103.66 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -218991.7325136824, delta loss: 87.62 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00, 

Accuracy: 80.77, Likelihood Estimation: -205804.04079047413, delta loss: 205804.04 Validation accuracy: 73.08% at iter=4000
Accuracy: 95.19, Likelihood Estimation: -205755.90851877464, delta loss: 48.13 Validation accuracy: 82.69% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -217935.8133380998, delta loss: 12179.90 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218162.6816313872, delta loss: 226.87 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -218286.77680540347, delta loss: 124.10 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -218369.15513288704, delta loss: 82.38 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -218429.34928488146, delta loss: 60.19 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -218476.28267102927, delta loss: 46.93 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00, Lik

Accuracy: 88.46, Likelihood Estimation: -190626.41212307755, delta loss: 190626.41 Validation accuracy: 82.69% at iter=4000
Accuracy: 75.00, Likelihood Estimation: -197042.3750006042, delta loss: 6415.96 Validation accuracy: 82.69% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -218788.7808050023, delta loss: 21746.41 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218978.0555037353, delta loss: 189.27 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -219038.79511963765, delta loss: 60.74 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -219048.17631734803, delta loss: 9.38 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -219029.17426753492, delta loss: 19.00 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -218992.10891253, delta loss: 37.07 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00, Likelih

Accuracy: 75.96, Likelihood Estimation: -203303.9043791868, delta loss: 203303.90 Validation accuracy: 71.15% at iter=4000
Accuracy: 88.46, Likelihood Estimation: -210150.63489406684, delta loss: 6846.73 Validation accuracy: 69.23% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -215312.7134714154, delta loss: 5162.08 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -214656.8520354784, delta loss: 655.86 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Likelihood Estimation: -213753.38888138533, delta loss: 903.46 Validation accuracy: 78.85% at iter=20000
Accuracy: 100.00, Likelihood Estimation: -212777.26847831663, delta loss: 976.12 Validation accuracy: 78.85% at iter=24000
Accuracy: 100.00, Likelihood Estimation: -211775.0806277074, delta loss: 1002.19 Validation accuracy: 78.85% at iter=28000
Accuracy: 100.00, Likelihood Estimation: -210759.7531493037, delta loss: 1015.33 Validation accuracy: 78.85% at iter=32000
Accuracy: 100.00, 

Accuracy: 84.62, Likelihood Estimation: -179088.68632668373, delta loss: 179088.69 Validation accuracy: 69.23% at iter=4000
Accuracy: 94.23, Likelihood Estimation: -180018.19063555406, delta loss: 929.50 Validation accuracy: 76.92% at iter=8000
Accuracy: 96.15, Likelihood Estimation: -191854.31875206812, delta loss: 11836.13 Validation accuracy: 76.92% at iter=12000
Accuracy: 95.19, Likelihood Estimation: -198346.50925488226, delta loss: 6492.19 Validation accuracy: 78.85% at iter=16000
Accuracy: 91.35, Likelihood Estimation: -200751.71127896468, delta loss: 2405.20 Validation accuracy: 80.77% at iter=20000
Accuracy: 97.12, Likelihood Estimation: -202718.60113455472, delta loss: 1966.89 Validation accuracy: 80.77% at iter=24000
Accuracy: 97.12, Likelihood Estimation: -205071.94890122282, delta loss: 2353.35 Validation accuracy: 76.92% at iter=28000
Accuracy: 96.15, Likelihood Estimation: -205319.44887349918, delta loss: 247.50 Validation accuracy: 76.92% at iter=32000
Accuracy: 86.54, 

In [47]:
epochs = 100000
lbd = [10.0**i for i in np.arange(-3,0)]
for c in lbd:
    print("Lambda =  ",c)
    clf = Logistic_regression_L2(1, c, data)
    prev_l= 0
    for i in tqdm(range(epochs)):
        clf.update_step(data)
        if(i%4000==0 and i>1000):
            print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {:.2f}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)),end=' ')
            print("Validation accuracy: {:.2f}% at iter={}".format(100*clf.accuracy(valid_data), i))
            # print(clf.w[:10])
            if(abs(clf.MLE(data)-prev_l)<1 or i==epochs-1):
                print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
                print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
                print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
                break
            prev_l = clf.MLE(data)
        if(i==epochs-1):
            print("delta loss: {} with MLE: {}".format(abs(clf.MLE(data)-prev_l), clf.MLE(data)))
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))

Lambda =   0.001


Accuracy: 70.19, Likelihood Estimation: -131664.23516570873, delta loss: 131664.24 Validation accuracy: 65.38% at iter=4000
Accuracy: 71.15, Likelihood Estimation: -110260.96893646811, delta loss: 21403.27 Validation accuracy: 78.85% at iter=8000
Accuracy: 70.19, Likelihood Estimation: -134734.2369751601, delta loss: 24473.27 Validation accuracy: 63.46% at iter=12000
Accuracy: 71.15, Likelihood Estimation: -111749.00041877429, delta loss: 22985.24 Validation accuracy: 78.85% at iter=16000
Accuracy: 69.23, Likelihood Estimation: -134595.36405681062, delta loss: 22846.36 Validation accuracy: 65.38% at iter=20000
Accuracy: 66.35, Likelihood Estimation: -138571.13368803958, delta loss: 3975.77 Validation accuracy: 61.54% at iter=24000
Accuracy: 69.23, Likelihood Estimation: -135890.66085336046, delta loss: 2680.47 Validation accuracy: 63.46% at iter=28000
Accuracy: 73.08, Likelihood Estimation: -111773.67877289144, delta loss: 24116.98 Validation accuracy: 78.85% at iter=32000
Accuracy: 84

Accuracy: 59.62, Likelihood Estimation: -107822.98316111762, delta loss: 107822.98 Validation accuracy: 67.31% at iter=4000
Accuracy: 61.54, Likelihood Estimation: -108372.51797240964, delta loss: 549.53 Validation accuracy: 67.31% at iter=8000
Accuracy: 61.54, Likelihood Estimation: -108498.02782875097, delta loss: 125.51 Validation accuracy: 67.31% at iter=12000
Accuracy: 61.54, Likelihood Estimation: -108142.05368648042, delta loss: 355.97 Validation accuracy: 67.31% at iter=16000
Accuracy: 61.54, Likelihood Estimation: -108157.12164361539, delta loss: 15.07 Validation accuracy: 67.31% at iter=20000
Accuracy: 61.54, Likelihood Estimation: -108367.02568887714, delta loss: 209.90 Validation accuracy: 67.31% at iter=24000
Accuracy: 59.62, Likelihood Estimation: -107723.03651721257, delta loss: 643.99 Validation accuracy: 67.31% at iter=28000
Accuracy: 61.54, Likelihood Estimation: -108614.46196970528, delta loss: 891.43 Validation accuracy: 69.23% at iter=32000
Accuracy: 59.62, Likelih

Accuracy: 50.96, Likelihood Estimation: 68636.408830876, delta loss: 68636.41 Validation accuracy: 61.54% at iter=4000
Accuracy: 50.96, Likelihood Estimation: 68636.40883106658, delta loss: 0.00 Validation accuracy: 61.54% at iter=8000
converged at iteration :8001 where delta loss: 1.9058643374592066e-07 with MLE: 68636.40883106658
Accuracy on Valid data: 61.54%
Accuracy on test data: 50.00%


In [52]:
epochs = 10000
lbd = [0.1]
for c in lbd:
    print("Lambda =  ",c)
    clf = Logistic_regression_L2(1, c, data)
    prev_l= 0
    for i in tqdm(range(epochs)):
        clf.update_step(data)
        if(i%4000==0):
            print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {:.2f}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)),end=' ')
            print("Validation accuracy: {:.2f}% at iter={}".format(100*clf.accuracy(valid_data), i))
            # print(clf.w[:10])
            if(clf.accuracy(data)==1 or i==epochs-1):
                print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
                print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
                print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
                break
            prev_l = clf.MLE(data)
        if(i==epochs-1):
            print("delta loss: {} with MLE: {}".format(abs(clf.MLE(data)-prev_l), clf.MLE(data)))
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))

Lambda =   0.1


Accuracy: 53.85, Likelihood Estimation: 241.54867760966792, delta loss: 241.55 Validation accuracy: 63.46% at iter=0
Accuracy: 50.96, Likelihood Estimation: 68636.408830876, delta loss: 68394.86 Validation accuracy: 61.54% at iter=4000
Accuracy: 50.96, Likelihood Estimation: 68636.40883106658, delta loss: 0.00 Validation accuracy: 61.54% at iter=8000
delta loss: 28705.505236741592 with MLE: 39930.90359432499
Accuracy on Valid data: 42.31%
Accuracy on test data: 50.00%



In [55]:
# best choice of lambda is 0.0001 and we train upto iteration 96000
clf = Logistic_regression_L2(1, 0.0001, data)
for i in tqdm(range(96000+1)):
    clf.update_step(data)

print("Accuracy on Train data: {:.2f}%".format(100*clf.accuracy(data)))
print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
print("corresponding clf weights: ", clf.w)
print("bias: ",clf.b)


Accuracy on Train data: 88.46%
Accuracy on Valid data: 86.54%
Accuracy on test data: 80.77%
corresponding clf weights:  [ 591.3556212   316.36414215 -306.3623457   495.13601673    5.99406324
 -138.43976991 -286.13298772 -800.58871579  853.47402031   -6.69099956
   96.11585241  164.84865863  170.22929247  -73.73582206  165.78239788
 -141.67301221 -598.40145337  334.18767941   97.248365    202.26481577
 -191.94770888  181.34857627   88.55860829   42.19218097   45.58287744
  -93.40057922 -173.67068912  157.48687145 -112.36390347  353.94279922
 -551.62586499  341.16239797   10.22308614 -354.34606383  324.04341947
 -396.19452055   45.26259922   81.14236829   28.38205235 -362.83754239
  369.15973157 -272.81208516  426.18427215  130.27222056   75.76020964
 -140.2277296   324.25053054  701.89673427  434.76444949 -265.66199812
   88.36461019  156.82662288   92.07802354  107.7711833   -45.71755983
   45.95899201  -39.19662311  165.45888245  171.11687335  150.56104425]
bias:  -231.23736688859256

# Logistic L1

In [0]:
class Logistic_regression_L1:
    def __init__(self, lr, lbd, data):
        self.w = np.zeros(data.shape[1]-1)
        self.b = 0
        self.lbd = lbd
        self.lr = lr

    def P(self, x, y):
        exp = np.exp(np.dot(x,self.w)+self.b)*y/(1+np.exp(np.dot(x,self.w)+self.b))
        return ((1-y)/2)+exp

    def predict(self, x):
        p_n1 = self.P(x,-1)
        p_p1 = self.P(x, 1)
        p = p_p1>=p_n1
        p = np.array(p).astype(np.int32)
        return p*2-1
    
    def accuracy(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        p = self.predict(x)
        # print(p)
        p = p*y
        return p[p>0].shape[0]/p.shape[0]

    def MLE(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        return np.sum(((y+1)/2)*np.dot(x,self.w+self.b) - np.log(1+np.exp(np.dot(x,self.w)+self.b))) - np.sum(np.absolute(self.w))*self.lbd

    def update_step(self, data):
        x = np.array(data[data.columns[:-1]])
        y = np.array(data['class'])
        delta_b = np.sum(( (y+1)/2 ) - self.P( x, np.ones(x.shape[0]) ))
        delta_w = (( (y+1)/2 ) - self.P( x, np.ones(x.shape[0]) )) @ x 
        self.w = self.w + (delta_w  - self.lbd * np.sign(self.w)) * self.lr
        self.b = self.b + delta_b*self.lr

In [42]:
epochs = 100000
for c in [0.01]:
    print("Lambda =  ",c)
    clf = Logistic_regression_L1(0.01, c, data)
    for i in tqdm(range(epochs)):
        clf.update_step(data)
        if(i%4000==0):
            print("Accuracy: {:.2f}, Likelihood Estimation: {:.4f}".format(100*clf.accuracy(data), clf.MLE(data)))
            # print(clf.w[:10])
        if(clf.accuracy(data)==1):
            print("converged at iteration : ",i+1)
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on Test data: {:.2f}%".format(100*clf.accuracy(test_data)))
            break

Lambda =   0.01


Accuracy: 53.85, Likelihood Estimation: -62.2771
Accuracy: 93.27, Likelihood Estimation: -2945.1233
Accuracy: 92.31, Likelihood Estimation: -3843.0171
Accuracy: 95.19, Likelihood Estimation: -4320.3832
Accuracy: 96.15, Likelihood Estimation: -4678.0745
Accuracy: 96.15, Likelihood Estimation: -5030.8871
Accuracy: 96.15, Likelihood Estimation: -5375.1296
Accuracy: 97.12, Likelihood Estimation: -5697.2178
Accuracy: 97.12, Likelihood Estimation: -5993.0957
Accuracy: 98.08, Likelihood Estimation: -6263.9340
Accuracy: 98.08, Likelihood Estimation: -6512.3167
Accuracy: 98.08, Likelihood Estimation: -6703.9755
Accuracy: 99.04, Likelihood Estimation: -6898.0369
Accuracy: 99.04, Likelihood Estimation: -7087.7245
Accuracy: 99.04, Likelihood Estimation: -7268.3037
Accuracy: 99.04, Likelihood Estimation: -7439.2107
Accuracy: 99.04, Likelihood Estimation: -7591.7155
Accuracy: 99.04, Likelihood Estimation: -7733.6153
Accuracy: 99.04, Likelihood Estimation: -7869.0294
Accuracy: 99.04, Likelihood Estim

In [59]:
epochs = 100000
lbd = [10.0**i for i in np.arange(-12,-1)]
for c in lbd:
    print("Lambda =  ",c)
    clf = Logistic_regression_L1(1, c, data)
    prev_l= 0
    for i in tqdm(range(epochs)):
        clf.update_step(data)
        if(i%2000==0 and i>1000):
            print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {:.2f}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)),end=' ')
            print("Validation accuracy: {:.2f}% at iter={}".format(100*clf.accuracy(valid_data), i))
            # print(clf.w[:10])
            if(abs(clf.MLE(data)-prev_l)<1 or i==epochs-1):
                print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
                print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
                print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
                break
            prev_l = clf.MLE(data)
        if(i==epochs-1):
            print("delta loss: {} with MLE: {}".format(abs(clf.MLE(data)-prev_l), clf.MLE(data)))
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))

Lambda =   1e-12


Accuracy: 76.92, Likelihood Estimation: -162778.8278686454, delta loss: 162778.83 Validation accuracy: 75.00% at iter=2000
Accuracy: 72.12, Likelihood Estimation: -209341.03042777537, delta loss: 46562.20 Validation accuracy: 63.46% at iter=4000
Accuracy: 87.50, Likelihood Estimation: -200662.0747262077, delta loss: 8678.96 Validation accuracy: 86.54% at iter=6000
Accuracy: 96.15, Likelihood Estimation: -211904.75346977456, delta loss: 11242.68 Validation accuracy: 71.15% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -217848.9759884027, delta loss: 5944.22 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -218235.7130023293, delta loss: 386.74 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218443.03818081119, delta loss: 207.33 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -218597.6285697456, delta loss: 154.59 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, Li

Accuracy: 82.69, Likelihood Estimation: -148896.58200152608, delta loss: 148896.58 Validation accuracy: 80.77% at iter=2000
Accuracy: 86.54, Likelihood Estimation: -199496.09466905624, delta loss: 50599.51 Validation accuracy: 71.15% at iter=4000
Accuracy: 96.15, Likelihood Estimation: -203936.96912158877, delta loss: 4440.87 Validation accuracy: 76.92% at iter=6000
Accuracy: 86.54, Likelihood Estimation: -202327.9464244007, delta loss: 1609.02 Validation accuracy: 86.54% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -217951.6730772428, delta loss: 15623.73 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -218258.2000071077, delta loss: 306.53 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218428.70427519295, delta loss: 170.50 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -218553.3470838096, delta loss: 124.64 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, L

Accuracy: 78.85, Likelihood Estimation: -162942.1974771629, delta loss: 162942.20 Validation accuracy: 71.15% at iter=2000
Accuracy: 87.50, Likelihood Estimation: -188732.0753499146, delta loss: 25789.88 Validation accuracy: 84.62% at iter=4000
Accuracy: 83.65, Likelihood Estimation: -210338.9120329936, delta loss: 21606.84 Validation accuracy: 67.31% at iter=6000
Accuracy: 94.23, Likelihood Estimation: -204367.24908957334, delta loss: 5971.66 Validation accuracy: 82.69% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -217350.03691196057, delta loss: 12982.79 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -217718.8193923158, delta loss: 368.78 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -217901.43002938374, delta loss: 182.61 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -218025.53835497788, delta loss: 124.11 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, 

Accuracy: 75.96, Likelihood Estimation: -142642.21390973567, delta loss: 142642.21 Validation accuracy: 80.77% at iter=2000
Accuracy: 89.42, Likelihood Estimation: -196146.7569334716, delta loss: 53504.54 Validation accuracy: 75.00% at iter=4000
Accuracy: 89.42, Likelihood Estimation: -200780.7723292465, delta loss: 4634.02 Validation accuracy: 84.62% at iter=6000
Accuracy: 87.50, Likelihood Estimation: -215731.6553146731, delta loss: 14950.88 Validation accuracy: 69.23% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -219235.90035955852, delta loss: 3504.25 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -219605.24817950433, delta loss: 369.35 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -219798.96460778828, delta loss: 193.72 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -219938.62458569172, delta loss: 139.66 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, 

...

Accuracy: 76.92, Likelihood Estimation: -140580.89746494684, delta loss: 140580.90 Validation accuracy: 80.77% at iter=2000
Accuracy: 88.46, Likelihood Estimation: -192015.5590635301, delta loss: 51434.66 Validation accuracy: 82.69% at iter=4000
Accuracy: 97.12, Likelihood Estimation: -205816.91436852713, delta loss: 13801.36 Validation accuracy: 76.92% at iter=6000
Accuracy: 95.19, Likelihood Estimation: -206188.71758897332, delta loss: 371.80 Validation accuracy: 82.69% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -219183.2344696461, delta loss: 12994.52 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -219499.04381138238, delta loss: 315.81 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -219654.8616978628, delta loss: 155.82 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -219764.16074091152, delta loss: 109.30 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, 

Accuracy: 68.27, Likelihood Estimation: -167729.06270083485, delta loss: 167729.06 Validation accuracy: 63.46% at iter=2000
Accuracy: 73.08, Likelihood Estimation: -182328.21439982107, delta loss: 14599.15 Validation accuracy: 80.77% at iter=4000
Accuracy: 95.19, Likelihood Estimation: -206608.45675007062, delta loss: 24280.24 Validation accuracy: 76.92% at iter=6000
Accuracy: 95.19, Likelihood Estimation: -206392.50699477963, delta loss: 215.95 Validation accuracy: 82.69% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -218833.69546198298, delta loss: 12441.19 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -219235.91133894553, delta loss: 402.22 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -219425.1453544763, delta loss: 189.23 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -219550.72932785153, delta loss: 125.58 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00

Accuracy: 68.27, Likelihood Estimation: -167902.11157447827, delta loss: 167902.11 Validation accuracy: 63.46% at iter=2000
Accuracy: 85.58, Likelihood Estimation: -202188.1907575389, delta loss: 34286.08 Validation accuracy: 71.15% at iter=4000
Accuracy: 75.00, Likelihood Estimation: -216336.34560884358, delta loss: 14148.15 Validation accuracy: 71.15% at iter=6000
Accuracy: 97.12, Likelihood Estimation: -210014.49309594443, delta loss: 6321.85 Validation accuracy: 75.00% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -217974.5045186214, delta loss: 7960.01 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -218257.91249136536, delta loss: 283.41 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -218383.91610442288, delta loss: 126.00 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -218456.11390324135, delta loss: 72.20 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00, 

Accuracy: 85.58, Likelihood Estimation: -155940.8745433505, delta loss: 155940.87 Validation accuracy: 71.15% at iter=2000
Accuracy: 70.19, Likelihood Estimation: -206393.78412825786, delta loss: 50452.91 Validation accuracy: 63.46% at iter=4000
Accuracy: 92.31, Likelihood Estimation: -201262.94486869406, delta loss: 5130.84 Validation accuracy: 82.69% at iter=6000
Accuracy: 97.12, Likelihood Estimation: -203832.28291540084, delta loss: 2569.34 Validation accuracy: 80.77% at iter=8000
Accuracy: 100.00, Likelihood Estimation: -218202.02692795324, delta loss: 14369.74 Validation accuracy: 78.85% at iter=10000
Accuracy: 100.00, Likelihood Estimation: -218129.71297597856, delta loss: 72.31 Validation accuracy: 78.85% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -217799.46289388783, delta loss: 330.25 Validation accuracy: 78.85% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -217415.57212575758, delta loss: 383.89 Validation accuracy: 78.85% at iter=16000
Accuracy: 100.00,

Accuracy: 92.31, Likelihood Estimation: -150870.04355183718, delta loss: 150870.04 Validation accuracy: 76.92% at iter=2000
Accuracy: 87.50, Likelihood Estimation: -178605.4283608487, delta loss: 27735.38 Validation accuracy: 84.62% at iter=4000
Accuracy: 86.54, Likelihood Estimation: -188909.928163875, delta loss: 10304.50 Validation accuracy: 69.23% at iter=6000
Accuracy: 94.23, Likelihood Estimation: -192991.91096021954, delta loss: 4081.98 Validation accuracy: 82.69% at iter=8000
Accuracy: 98.08, Likelihood Estimation: -205577.94052795594, delta loss: 12586.03 Validation accuracy: 80.77% at iter=10000
Accuracy: 98.08, Likelihood Estimation: -219581.7867943095, delta loss: 14003.85 Validation accuracy: 80.77% at iter=12000
Accuracy: 100.00, Likelihood Estimation: -216346.61592960326, delta loss: 3235.17 Validation accuracy: 80.77% at iter=14000
Accuracy: 100.00, Likelihood Estimation: -211081.2860587497, delta loss: 5265.33 Validation accuracy: 80.77% at iter=16000
Accuracy: 100.00,

In [60]:
epochs = 100000
lbd = [10.0**i for i in np.arange(-1,1)]
for c in lbd:
    print("Lambda =  ",c)
    clf = Logistic_regression_L1(1, c, data)
    prev_l= 0
    for i in tqdm(range(epochs)):
        clf.update_step(data)
        if(i%2000==0 and i>1000):
            print("Accuracy: {:.2f}, Likelihood Estimation: {}, delta loss: {:.2f}".format(100*clf.accuracy(data), clf.MLE(data), abs(clf.MLE(data)-prev_l)),end=' ')
            print("Validation accuracy: {:.2f}% at iter={}".format(100*clf.accuracy(valid_data), i))
            # print(clf.w[:10])
            if(abs(clf.MLE(data)-prev_l)<1 or i==epochs-1):
                print("converged at iteration :{} where delta loss: {} with MLE: {}".format(i+1,abs(clf.MLE(data)-prev_l), clf.MLE(data)))
                print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
                print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
                break
            prev_l = clf.MLE(data)
        if(i==epochs-1):
            print("delta loss: {} with MLE: {}".format(abs(clf.MLE(data)-prev_l), clf.MLE(data)))
            print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
            print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))

Lambda =   0.1


Accuracy: 82.69, Likelihood Estimation: -110375.19133505556, delta loss: 110375.19 Validation accuracy: 78.85% at iter=2000
Accuracy: 88.46, Likelihood Estimation: -114837.31374807906, delta loss: 4462.12 Validation accuracy: 80.77% at iter=4000
Accuracy: 90.38, Likelihood Estimation: -126142.09043689101, delta loss: 11304.78 Validation accuracy: 76.92% at iter=6000
Accuracy: 87.50, Likelihood Estimation: -125231.63259646333, delta loss: 910.46 Validation accuracy: 86.54% at iter=8000
Accuracy: 72.12, Likelihood Estimation: -119256.78845508293, delta loss: 5974.84 Validation accuracy: 78.85% at iter=10000
Accuracy: 68.27, Likelihood Estimation: -151134.54721610693, delta loss: 31877.76 Validation accuracy: 65.38% at iter=12000
Accuracy: 66.35, Likelihood Estimation: -157463.08017499585, delta loss: 6328.53 Validation accuracy: 65.38% at iter=14000
Accuracy: 81.73, Likelihood Estimation: -146219.61415998254, delta loss: 11243.47 Validation accuracy: 71.15% at iter=16000
Accuracy: 81.73,

Accuracy: 61.54, Likelihood Estimation: -142125.46274584314, delta loss: 142125.46 Validation accuracy: 73.08% at iter=2000
Accuracy: 56.73, Likelihood Estimation: -141530.47571349438, delta loss: 594.99 Validation accuracy: 67.31% at iter=4000
Accuracy: 58.65, Likelihood Estimation: -145663.3995516308, delta loss: 4132.92 Validation accuracy: 67.31% at iter=6000
Accuracy: 64.42, Likelihood Estimation: -145640.85035809755, delta loss: 22.55 Validation accuracy: 75.00% at iter=8000
Accuracy: 58.65, Likelihood Estimation: -146933.22519033906, delta loss: 1292.37 Validation accuracy: 67.31% at iter=10000
Accuracy: 60.58, Likelihood Estimation: -147839.8762642594, delta loss: 906.65 Validation accuracy: 67.31% at iter=12000
Accuracy: 55.77, Likelihood Estimation: -146554.23503133157, delta loss: 1285.64 Validation accuracy: 67.31% at iter=14000
Accuracy: 63.46, Likelihood Estimation: -148588.73493650192, delta loss: 2034.50 Validation accuracy: 73.08% at iter=16000
Accuracy: 59.62, Likelih

In [74]:
clf1 = Logistic_regression_L1(1, 0.01, data)
for i in tqdm(range(46000+1)):
    clf1.update_step(data)
print("Accuracy on Train data: {:.2f}%".format(100*clf1.accuracy(data)))
print("Accuracy on Valid data: {:.2f}%".format(100*clf1.accuracy(valid_data)))
print("Accuracy on test data: {:.2f}%".format(100*clf1.accuracy(test_data)))
print("corresponding clf weights: ", clf1.w)
print("bias: ",clf1.b)


Accuracy on Train data: 100.00%
Accuracy on Valid data: 82.69%
Accuracy on test data: 78.85%
corresponding clf weights:  [ 7.78169708e+02  3.22992636e+02 -1.54306964e+02  5.98803725e+02
 -3.01229787e-03 -1.23401688e+02 -1.10645691e+02 -1.12803077e+03
  1.01945163e+03  5.77385543e-03  5.75654517e+01  1.12412542e+02
  9.59671763e+01 -5.70293835e+01  7.47855964e+01  1.14798797e-02
 -6.91054700e+02  4.04830917e+02  1.47435659e+02  1.21229308e+02
 -5.86202663e+01  5.82629635e+01  1.17414908e+02  3.19524578e+01
  4.47633210e+00  1.40511136e-02 -1.77730096e+02  7.83397458e+01
 -2.12759084e+01  3.67694685e+02 -6.44771148e+02  3.71977648e+02
 -2.87363173e-03 -4.62273462e+02  3.95705328e+02 -3.68324391e+02
  2.34131122e+01  7.90193424e+00 -2.27342948e-03 -2.59839575e+02
  3.67964324e+02 -2.57176153e+02  5.10666731e+02  4.35365505e-05
 -4.22933978e-03 -1.37512347e+01  1.28607622e+02  8.95399703e+02
  4.05547696e+02 -9.07249045e+01  6.31185133e-03 -4.16187175e-03
 -8.16808824e-03 -1.49195375e-03 

In [75]:
print("number of values close to zero {}".format(clf1.w[(clf1.w*np.sign(clf1.w))<0.1].shape[0]))


number of values close to zero 18


In [65]:
clf.w[(clf.w*np.sign(clf.w))<1]

array([ 0.11123082,  0.98924447,  0.64485833,  0.88911564,  0.34722295,
       -0.07555033,  0.10406788, -0.08954343])

In [68]:
# best choice of lambda is 0.0001 and we train upto iteration 96000
clf2 = Logistic_regression_L2(1, 0.0001, data)
for i in tqdm(range(96000+1)):
    clf.update_step(data)

print("Accuracy on Train data: {:.2f}%".format(100*clf.accuracy(data)))
print("Accuracy on Valid data: {:.2f}%".format(100*clf.accuracy(valid_data)))
print("Accuracy on test data: {:.2f}%".format(100*clf.accuracy(test_data)))
print("corresponding clf weights: ", clf.w)
print("bias: ",clf.b)


Accuracy on Train data: 92.31%
Accuracy on Valid data: 80.77%
Accuracy on test data: 82.69%
corresponding clf weights:  [ 592.96062515  315.87957248 -310.82975084  498.15932478    6.24849555
 -136.61612088 -285.03086667 -799.58505206  859.43066026    1.90419633
   96.01730739  158.5796738   161.28380896  -83.43111763  159.70608535
 -139.42512328 -600.70981838  336.78368974   94.42836018  200.44410758
 -192.42890159  182.60045769   92.34288213   39.16273871   43.85084203
  -89.79259272 -176.59558539  152.74672202 -113.62893084  359.33489128
 -552.40204954  338.25138111    1.26842124 -358.06712811  326.65802874
 -399.15733497   48.22490707   81.24105801   23.5853504  -373.05966853
  364.99413136 -281.28286652  424.695349    128.90759397   71.07170637
 -139.67517729  325.27396609  699.72524222  435.22629754 -267.93469506
   88.33136325  157.11378688   91.91456464  107.95732816  -45.38802998
   45.69999715  -40.09392153  166.78394835  170.92872594  150.88780101]
bias:  -238.85706210984867